In [ ]:
# Task 2 - Technical Indicator Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import datetime as dt
import re
import talib
import matplotlib as mpl

mpl.rcParams['figure.max_open_warning'] = 50  # or any number you prefer
# Load stock price data
nvda_df = pd.read_csv("../data/NVDA_historical_data.csv")
tsla_df = pd.read_csv("../data/TSLA_historical_data.csv")
aapl_df = pd.read_csv("../data/AAPL_historical_data.csv")
meta_df = pd.read_csv("../data/META_historical_data.csv")
msft_df = pd.read_csv("../data/MSFT_historical_data.csv")
goog_df = pd.read_csv("../data/GOOG_historical_data.csv")
amzn_df = pd.read_csv("../data/AMZN_historical_data.csv")

stock_data = {
    'NVDA': nvda_df,
    'TSLA': tsla_df,
    'AAPL': aapl_df,
    'META': meta_df,
    'MSFT': msft_df,
    'GOOG': goog_df,
    'AMZN': amzn_df
}

# Clean and prepare data
for name, df in stock_data.items():
    df.columns = df.columns.str.strip().str.lower()
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Calculate technical indicators
    df['SMA_20'] = talib.SMA(df['close'], timeperiod=20)
    df['EMA_20'] = talib.EMA(df['close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    macd, macdsignal, macdhist = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACD_signal'] = macdsignal
    df['MACD_hist'] = macdhist

    # Plot close + SMA + EMA
    plt.figure(figsize=(12,6))
    plt.plot(df['date'], df['close'], label='Close')
    plt.plot(df['date'], df['SMA_20'], label='SMA 20')
    plt.plot(df['date'], df['EMA_20'], label='EMA 20')
    plt.title(f'{name} Price with SMA & EMA')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{name.lower()}_sma_ema.png")

    # Plot RSI
    plt.figure(figsize=(10,4))
    plt.plot(df['date'], df['RSI'], color='purple')
    plt.axhline(70, color='red', linestyle='--')
    plt.axhline(30, color='green', linestyle='--')
    plt.title(f'{name} RSI')
    plt.tight_layout()
    plt.savefig(f"{name.lower()}_rsi.png")

    # Plot MACD
    plt.figure(figsize=(12,4))
    plt.plot(df['date'], df['MACD'], label='MACD')
    plt.plot(df['date'], df['MACD_signal'], label='Signal')
    plt.bar(df['date'], df['MACD_hist'], label='Histogram')
    plt.title(f'{name} MACD')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{name.lower()}_macd.png")

    # Save processed data
    df.to_csv(f"../data/{name.lower()}_indicators.csv", index=False)

print("Task 2 complete: Indicators calculated and charts saved.")